In [1]:
!pip install optuna

In [2]:
import optuna
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [3]:
import xgboost as xgb

In [4]:
data = pd.read_csv("https://raw.githubusercontent.com/Chandrakant817/Admission-Prediction/main/Admission_Prediction.csv")
data.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337.0,118.0,4.0,4.5,4.5,9.65,1,0.92
1,2,324.0,107.0,4.0,4.0,4.5,8.87,1,0.76
2,3,NaN,104.0,3.0,3.0,3.5,8.00,1,0.72
3,4,322.0,110.0,3.0,3.5,2.5,8.67,1,0.80
4,5,314.0,103.0,2.0,2.0,3.0,8.21,0,0.65


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Serial No.         500 non-null    int64  
 1   GRE Score          485 non-null    float64
 2   TOEFL Score        490 non-null    float64
 3   University Rating  485 non-null    float64
 4   SOP                500 non-null    float64
 5   LOR                500 non-null    float64
 6   CGPA               500 non-null    float64
 7   Research           500 non-null    int64  
 8   Chance of Admit    500 non-null    float64
dtypes: float64(7), int64(2)
memory usage: 35.3 KB


In [6]:
data.describe()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,500.000000,485.000000,490.000000,485.000000,500.000000,500.00000,500.000000,500.000000,500.00000
mean,250.500000,316.558763,107.187755,3.121649,3.374000,3.48400,8.576440,0.560000,0.72174
std,144.481833,11.274704,6.112899,1.146160,0.991004,0.92545,0.604813,0.496884,0.14114
min,1.000000,290.000000,92.000000,1.000000,1.000000,1.00000,6.800000,0.000000,0.34000
25%,125.750000,308.000000,103.000000,2.000000,2.500000,3.00000,8.127500,0.000000,0.63000
50%,250.500000,317.000000,107.000000,3.000000,3.500000,3.50000,8.560000,1.000000,0.72000
75%,375.250000,325.000000,112.000000,4.000000,4.000000,4.00000,9.040000,1.000000,0.82000
max,500.000000,340.000000,120.000000,5.000000,5.000000,5.00000,9.920000,1.000000,0.97000


In [7]:
data.isnull().sum()

Serial No.            0
GRE Score            15
TOEFL Score          10
University Rating    15
SOP                   0
LOR                   0
CGPA                  0
Research              0
Chance of Admit       0
dtype: int64

In [8]:
data.isnull().sum().sum()

40

In [9]:
data['GRE Score'] = data['GRE Score'].fillna(data['GRE Score'].median())
data['TOEFL Score'] = data['TOEFL Score'].fillna(data['TOEFL Score'].median())
data['University Rating'] = data['University Rating'].fillna(data['University Rating'].median())

In [10]:
data.isnull().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

In [11]:
data.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337.0,118.0,4.0,4.5,4.5,9.65,1,0.92
1,2,324.0,107.0,4.0,4.0,4.5,8.87,1,0.76
2,3,317.0,104.0,3.0,3.0,3.5,8.00,1,0.72
3,4,322.0,110.0,3.0,3.5,2.5,8.67,1,0.80
4,5,314.0,103.0,2.0,2.0,3.0,8.21,0,0.65


In [12]:
X = data.drop(['Serial No.','Chance of Admit'],axis=1)
X

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,337.0,118.0,4.0,4.5,4.5,9.65,1
1,324.0,107.0,4.0,4.0,4.5,8.87,1
2,317.0,104.0,3.0,3.0,3.5,8.00,1
3,322.0,110.0,3.0,3.5,2.5,8.67,1
4,314.0,103.0,2.0,2.0,3.0,8.21,0
...,...,...,...,...,...,...,...
495,332.0,108.0,5.0,4.5,4.0,9.02,1
496,337.0,117.0,5.0,5.0,5.0,9.87,1
497,330.0,120.0,5.0,4.5,5.0,9.56,1
498,312.0,103.0,4.0,4.0,5.0,8.43,0


In [13]:
y = data['Chance of Admit']
y

0      0.92
1      0.76
2      0.72
3      0.80
4      0.65
       ... 
495    0.87
496    0.96
497    0.93
498    0.73
499    0.84
Name: Chance of Admit, Length: 500, dtype: float64

In [14]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=25)

In [15]:
X_train

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
250,320.0,104.0,3.0,3.0,2.5,8.57,1
112,301.0,107.0,3.0,3.5,3.5,8.34,1
100,322.0,107.0,3.0,3.5,3.5,8.46,1
379,311.0,99.0,1.0,2.5,3.0,8.43,1
385,335.0,117.0,5.0,5.0,5.0,9.82,1
...,...,...,...,...,...,...,...
317,300.0,99.0,1.0,1.0,2.5,8.01,0
143,340.0,120.0,4.0,4.5,4.0,9.92,1
474,308.0,105.0,4.0,3.0,2.5,7.95,1
318,324.0,111.0,3.0,2.5,2.0,8.80,1


In [16]:
X_test

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
411,313.0,94.0,2.0,2.5,1.5,8.13,0
380,322.0,104.0,3.0,3.5,4.0,8.84,1
38,304.0,105.0,1.0,3.0,1.5,7.50,0
254,321.0,114.0,4.0,4.0,5.0,9.12,0
234,330.0,113.0,5.0,5.0,4.0,9.31,1
...,...,...,...,...,...,...,...
391,318.0,106.0,3.0,2.0,3.0,8.65,0
283,321.0,111.0,3.0,2.5,3.0,8.90,1
460,319.0,105.0,4.0,4.0,4.5,8.66,1
280,311.0,102.0,3.0,4.5,4.0,8.64,1


In [17]:
y_train

250    0.74
112    0.62
100    0.71
379    0.71
385    0.96
       ... 
317    0.58
143    0.97
474    0.67
318    0.79
132    0.71
Name: Chance of Admit, Length: 375, dtype: float64

In [18]:
y_test

411    0.56
380    0.78
38     0.52
254    0.85
234    0.91
       ... 
391    0.71
283    0.80
460    0.77
280    0.68
363    0.69
Name: Chance of Admit, Length: 125, dtype: float64

In [19]:
sc = StandardScaler()

In [20]:
X_train_sc = sc.fit_transform(X_train)
X_train_sc

array([[ 2.90198444e-01, -5.41014617e-01, -7.62029204e-02, ...,
        -1.02555725e+00, -1.32267264e-03,  9.10641693e-01],
       [-1.42855793e+00, -3.59179829e-02, -7.62029204e-02, ...,
         5.17087690e-02, -3.81591057e-01,  9.10641693e-01],
       [ 4.71120167e-01, -3.59179829e-02, -7.62029204e-02, ...,
         5.17087690e-02, -1.83190161e-01,  9.10641693e-01],
       ...,
       [-7.95331895e-01, -3.72649072e-01,  7.89739357e-01, ...,
        -1.02555725e+00, -1.02639397e+00,  9.10641693e-01],
       [ 6.52041890e-01,  6.37544196e-01, -7.62029204e-02, ...,
        -1.56419026e+00,  3.78945712e-01,  9.10641693e-01],
       [-7.04871034e-01, -3.72649072e-01,  1.65568163e+00, ...,
         5.17087690e-02, -1.78560807e-02, -1.09812675e+00]])

In [21]:
X.isnull().sum()

GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
dtype: int64

In [22]:
X_test_sc = sc.transform(X_test)

In [23]:
X_test

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
411,313.0,94.0,2.0,2.5,1.5,8.13,0
380,322.0,104.0,3.0,3.5,4.0,8.84,1
38,304.0,105.0,1.0,3.0,1.5,7.50,0
254,321.0,114.0,4.0,4.0,5.0,9.12,0
234,330.0,113.0,5.0,5.0,4.0,9.31,1
...,...,...,...,...,...,...,...
391,318.0,106.0,3.0,2.0,3.0,8.65,0
283,321.0,111.0,3.0,2.5,3.0,8.90,1
460,319.0,105.0,4.0,4.0,4.5,8.66,1
280,311.0,102.0,3.0,4.5,4.0,8.64,1


In [24]:
def objective(trial,data=X,target=y):
    train_x,test_x,train_y,test_y=train_test_split(data,target,test_size=0.25,random_state=25)
    param = {
        "tree_method":"auto",
        "lambda":trial.suggest_loguniform('lambda',1e-4,10.0),
        "alpha":trial.suggest_loguniform('alpha',1e-4,10.0),
        "colsample_bytree":trial.suggest_categorical('colsample_bytree',[.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
        "subsample":trial.suggest_categorical('subsample',[.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
        "learning_rate":trial.suggest_categorical('learning_rate',[0.00001,0.0003,0.008,0.02,0.01,1,8]),
        "n_estimators":300,
        "max_depth":trial.suggest_categorical('max_depth',[3,4,5,6,7,8,9,10,11,12]),
        "random_state":trial.suggest_categorical('random_state',[10,20,30,2000,3454,243123]),
        "min_child_weight":trial.suggest_int("min_child_weight",1,200)
    }
    model = xgb.XGBRegressor(**param)
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)],verbose=True)
    pred = model.predict(test_x)
    mse = mean_squared_error(test_y,pred)
    return mse

In [26]:
find_params = optuna.create_study()
find_params.optimize(objective,n_trials=10)

[I 2024-03-21 00:52:08,789] A new study created in memory with name: no-name-8603dbd4-48b1-445b-8e99-b0a611039940


[0]	validation_0-rmse:0.07615
[1]	validation_0-rmse:0.07509
[2]	validation_0-rmse:0.07531
[3]	validation_0-rmse:0.07520
[4]	validation_0-rmse:0.07529
[5]	validation_0-rmse:0.07530
[6]	validation_0-rmse:0.07380
[7]	validation_0-rmse:0.07363
[8]	validation_0-rmse:0.07363
[9]	validation_0-rmse:0.07363
[10]	validation_0-rmse:0.07363
[11]	validation_0-rmse:0.07363
[12]	validation_0-rmse:0.07363
[13]	validation_0-rmse:0.07363
[14]	validation_0-rmse:0.07363
[15]	validation_0-rmse:0.07363
[16]	validation_0-rmse:0.07359
[17]	validation_0-rmse:0.07359
[18]	validation_0-rmse:0.07359
[19]	validation_0-rmse:0.07359
[20]	validation_0-rmse:0.07359
[21]	validation_0-rmse:0.07359
[22]	validation_0-rmse:0.07359
[23]	validation_0-rmse:0.07359
[24]	validation_0-rmse:0.07359
[25]	validation_0-rmse:0.07359
[26]	validation_0-rmse:0.07359
[27]	validation_0-rmse:0.07359
[28]	validation_0-rmse:0.07359
[29]	validation_0-rmse:0.07359
[30]	validation_0-rmse:0.07359
[31]	validation_0-rmse:0.07359
[32]	validation_0-

[260]	validation_0-rmse:0.07359
[261]	validation_0-rmse:0.07359
[262]	validation_0-rmse:0.07359
[263]	validation_0-rmse:0.07359
[264]	validation_0-rmse:0.07359
[265]	validation_0-rmse:0.07359
[266]	validation_0-rmse:0.07359
[267]	validation_0-rmse:0.07359
[268]	validation_0-rmse:0.07359
[269]	validation_0-rmse:0.07359
[270]	validation_0-rmse:0.07359
[271]	validation_0-rmse:0.07359
[272]	validation_0-rmse:0.07359
[273]	validation_0-rmse:0.07359
[274]	validation_0-rmse:0.07359
[275]	validation_0-rmse:0.07359
[276]	validation_0-rmse:0.07359
[277]	validation_0-rmse:0.07359
[278]	validation_0-rmse:0.07359
[279]	validation_0-rmse:0.07359
[280]	validation_0-rmse:0.07359
[281]	validation_0-rmse:0.07359
[282]	validation_0-rmse:0.07359
[283]	validation_0-rmse:0.07359
[284]	validation_0-rmse:0.07359
[285]	validation_0-rmse:0.07359
[286]	validation_0-rmse:0.07359
[287]	validation_0-rmse:0.07359
[288]	validation_0-rmse:0.07359
[289]	validation_0-rmse:0.07359
[290]	validation_0-rmse:0.07359
[291]	va

/var/folders/31/h661wdqj5pzb01symws5rm_40000gn/T/ipykernel_28185/3119446367.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "lambda":trial.suggest_loguniform('lambda',1e-4,10.0),
/var/folders/31/h661wdqj5pzb01symws5rm_40000gn/T/ipykernel_28185/3119446367.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "alpha":trial.suggest_loguniform('alpha',1e-4,10.0),
[I 2024-03-21 00:52:08,976] Trial 0 finished with value: 0.005415183362299695 and parameters: {'lambda': 0.3365295847560206, 'alpha': 1.2408156018961747, 'colsample_bytree': 0.3, 'subsample': 1, 'learning_rate': 1, 'max_depth': 5, 'random_state': 20, 'min_child_weight': 19}. Best is trial 0 with value: 0.00541518

[0]	validation_0-rmse:0.26233
[1]	validation_0-rmse:0.25925
[2]	validation_0-rmse:0.25561
[3]	validation_0-rmse:0.25271
[4]	validation_0-rmse:0.24911
[5]	validation_0-rmse:0.24558
[6]	validation_0-rmse:0.24195
[7]	validation_0-rmse:0.23907
[8]	validation_0-rmse:0.23623
[9]	validation_0-rmse:0.23254
[10]	validation_0-rmse:0.22945
[11]	validation_0-rmse:0.22676
[12]	validation_0-rmse:0.22434
[13]	validation_0-rmse:0.22104
[14]	validation_0-rmse:0.21789
[15]	validation_0-rmse:0.21541
[16]	validation_0-rmse:0.21322
[17]	validation_0-rmse:0.21085
[18]	validation_0-rmse:0.20813
[19]	validation_0-rmse:0.20569
[20]	validation_0-rmse:0.20278
[21]	validation_0-rmse:0.19990
[22]	validation_0-rmse:0.19787
[23]	validation_0-rmse:0.19536
[24]	validation_0-rmse:0.19327


...

[I 2024-03-21 00:52:10,240] Trial 8 finished with value: 0.0705856 and parameters: {'lambda': 4.395816085667108, 'alpha': 0.014964545918772092, 'colsample_bytree': 0.2, 'subsample': 0.2, 'learning_rate': 1e-05, 'max_depth': 4, 'random_state': 20, 'min_child_weight': 171}. Best is trial 0 with value: 0.005415183362299695.


[0]	validation_0-rmse:0.11576
[1]	validation_0-rmse:0.08972
[2]	validation_0-rmse:0.08611
[3]	validation_0-rmse:0.08322
[4]	validation_0-rmse:0.08187
[5]	validation_0-rmse:0.08218
[6]	validation_0-rmse:0.08232
[7]	validation_0-rmse:0.07618
[8]	validation_0-rmse:0.07282
[9]	validation_0-rmse:0.07343
[10]	validation_0-rmse:0.07095
[11]	validation_0-rmse:0.07131
[12]	validation_0-rmse:0.07137
[13]	validation_0-rmse:0.07132
[14]	validation_0-rmse:0.07085
[15]	validation_0-rmse:0.07192
[16]	validation_0-rmse:0.07287
[17]	validation_0-rmse:0.07229
[18]	validation_0-rmse:0.07228
[19]	validation_0-rmse:0.07223
[20]	validation_0-rmse:0.07259
[21]	validation_0-rmse:0.07273
[22]	validation_0-rmse:0.07239
[23]	validation_0-rmse:0.07284
[24]	validation_0-rmse:0.07203
[25]	validation_0-rmse:0.06999
[26]	validation_0-rmse:0.06991
[27]	validation_0-rmse:0.06978
[28]	validation_0-rmse:0.06975
[29]	validation_0-rmse:0.06929
[30]	validation_0-rmse:0.06877
[31]	validation_0-rmse:0.06823
[32]	validation_0-

/var/folders/31/h661wdqj5pzb01symws5rm_40000gn/T/ipykernel_28185/3119446367.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "lambda":trial.suggest_loguniform('lambda',1e-4,10.0),
/var/folders/31/h661wdqj5pzb01symws5rm_40000gn/T/ipykernel_28185/3119446367.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "alpha":trial.suggest_loguniform('alpha',1e-4,10.0),


[83]	validation_0-rmse:0.06750
[84]	validation_0-rmse:0.06828
[85]	validation_0-rmse:0.06816
[86]	validation_0-rmse:0.06817
[87]	validation_0-rmse:0.06796
[88]	validation_0-rmse:0.06777
[89]	validation_0-rmse:0.06785
[90]	validation_0-rmse:0.06888
[91]	validation_0-rmse:0.06844
[92]	validation_0-rmse:0.06789
[93]	validation_0-rmse:0.06765
[94]	validation_0-rmse:0.06780
[95]	validation_0-rmse:0.06702
[96]	validation_0-rmse:0.06811
[97]	validation_0-rmse:0.06773
[98]	validation_0-rmse:0.06818
[99]	validation_0-rmse:0.06735
[100]	validation_0-rmse:0.06701
[101]	validation_0-rmse:0.06755
[102]	validation_0-rmse:0.06682
[103]	validation_0-rmse:0.06728
[104]	validation_0-rmse:0.06704
[105]	validation_0-rmse:0.06702
[106]	validation_0-rmse:0.06676
[107]	validation_0-rmse:0.06673
[108]	validation_0-rmse:0.06658
[109]	validation_0-rmse:0.06691
[110]	validation_0-rmse:0.06679
[111]	validation_0-rmse:0.06723
[112]	validation_0-rmse:0.06710
[113]	validation_0-rmse:0.06673
[114]	validation_0-rmse:0

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [27]:
 find_params.best_trial.params

{'lambda': 0.006455504392117833,
 'alpha': 0.00848362371862707,
 'colsample_bytree': 0.3,
 'subsample': 0.9,
 'learning_rate': 1,
 'max_depth': 11,
 'random_state': 3454,
 'min_child_weight': 107}

In [ ]:
!python --version

In [28]:
best_params = {'lambda': 0.006455504392117833,
 'alpha': 0.00848362371862707,
 'colsample_bytree': 0.3,
 'subsample': 0.9,
 'learning_rate': 1,
 'max_depth': 11,
 'random_state': 3454,
 'min_child_weight': 107}

In [29]:
model = xgb.XGBRegressor(**best_params)
model

XGBRegressor(alpha=0.00848362371862707, base_score=None, booster=None,
             callbacks=None, colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.3, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, gamma=None,
             gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, lambda=0.006455504392117833,
             learning_rate=1, max_bin=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=11, max_leaves=None,
             min_child_weight=107, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=3454, ...)

In [30]:
model.fit(X_train,y_train)

XGBRegressor(alpha=0.00848362371862707, base_score=0.5, booster='gbtree',
             callbacks=None, colsample_bylevel=1, colsample_bynode=1,
             colsample_bytree=0.3, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', lambda=0.006455504392117833,
             learning_rate=1, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=11, max_leaves=0, min_child_weight=107,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=3454, ...)

In [31]:
y_pred = model.predict(X_test)
y_pred

array([0.5602815 , 0.7477563 , 0.56923366, 0.86121505, 0.8927841 ,
       0.8385512 , 0.57907766, 0.88876283, 0.8806591 , 0.7138032 ,
       0.8927841 , 0.7971027 , 0.8316645 , 0.5661842 , 0.56020147,
       0.8115544 , 0.6094432 , 0.68189514, 0.5455051 , 0.77690274,
       0.88876283, 0.56754476, 0.8927841 , 0.85528105, 0.8927841 ,
       0.8283556 , 0.74609643, 0.7079953 , 0.8373631 , 0.56923366,
       0.5661842 , 0.5455051 , 0.6322827 , 0.86605555, 0.5689782 ,
       0.8927841 , 0.7146207 , 0.6970633 , 0.7625519 , 0.8246191 ,
       0.5455051 , 0.760445  , 0.6886767 , 0.8927841 , 0.56020147,
       0.8927841 , 0.6054219 , 0.854907  , 0.8927841 , 0.6333863 ,
       0.58338225, 0.88876283, 0.56923366, 0.8927841 , 0.7773094 ,
       0.60406137, 0.8927841 , 0.5455051 , 0.86909556, 0.58339757,
       0.5455051 , 0.8927841 , 0.67495704, 0.559474  , 0.7115605 ,
       0.597221  , 0.70028985, 0.5455051 , 0.56754476, 0.81381977,
       0.8667232 , 0.8594245 , 0.8927841 , 0.6948927 , 0.54550

In [32]:
from sklearn.metrics import r2_score
r2_score(y_true=y_test,y_pred=y_pred)

0.7845479296126721

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model2 = RandomForestRegressor()
model2.fit(X_train,y_train)
y_pred2 = model2.predict(X_test)
r2_score(y_test,y_pred2)